# Visual SLAM Colab Setup (Web + 3DGS Mapping)

This notebook sets up the repo, installs dependencies, downloads TUM, and runs the pipeline.

Update the repo URL below if needed.



In [ ]:
# --- Config ---
REPO_URL = "https://github.com/<YOUR_ORG>/visual-slam.git"  # TODO: replace
REPO_DIR = "/content/visual-slam"
DATASET_DIR = "/content/datasets/tum/fr3_office"

!nvidia-smi || true



In [ ]:
# --- System deps ---
!sudo apt-get update -y
!sudo apt-get install -y libeigen3-dev ninja-build



In [ ]:
# --- Clone repo ---
import os
import subprocess

if os.path.isdir(REPO_DIR):
    print("Repo already exists, skipping clone")
else:
    subprocess.run(["git", "clone", REPO_URL, REPO_DIR], check=True)

%cd {REPO_DIR}



In [ ]:
# --- Install Python deps ---
%pip install -r requirements.txt



In [ ]:
# --- Configure config for Colab ---
import yaml
from pathlib import Path

cfg_path = Path(REPO_DIR) / "configs" / "tum_fr3_office.yaml"
cfg = yaml.safe_load(cfg_path.read_text())

cfg["dataset"]["root"] = DATASET_DIR
# Default tracking mode is orb_pnp, no need to set droid
cfg["tracking"]["use_gt_init"] = False

cfg_path.write_text(yaml.safe_dump(cfg, sort_keys=False))
print(cfg_path.read_text())



In [ ]:
# --- Run SLAM (will download dataset if missing) ---
import subprocess

subprocess.run([
    "python",
    "run_slam.py",
    "--config",
    str(Path(REPO_DIR) / "configs" / "tum_fr3_office.yaml"),
], check=True)



In [ ]:
# --- Visualize final PLY ---
import subprocess

subprocess.run([
    "python",
    "view_ply.py",
    "--ply",
    str(Path(REPO_DIR) / "outputs" / "map.ply"),
], check=True)

